 <div class="alert alert-block alert-info">
    <b>Note:</b> 
    Interchange is in the process of replacing ParmEd in many workflows, but it still in an alpha testing phase. Our internal tests indicate it is reliable for many small-molecule systems, but it is not yet reliable for complex, multi-component systems and there are likely still rough edges throughout. Feedback is welcome on the <a href=https://github.com/openforcefield/openff-interchange/issues/>Interchange issue tracker.</a></div>

## Using OpenFF force fields in Amber and GROMACS

The Open Forcefield Toolkit can create parametrized `openmm.System` objects that can be natively simulated with OpenMM. This example shows the Interchange project can enable parallel workflows using Amber and GROMACS.

### Preparing an OpenFF Topology

We start by loading a PDB file containing one copy of ethanol and cyclohexane. Our goal is to create an OpenFF `Topology` object describing this system that we can parametrize with the SMIRNOFF-format "Sage" force field.

The two `Molecule` objects created from the SMILES strings can contain information such as partial charges and stereochemistry that is not included in an OpenMM topology. In this example, partial charges are not explicitly given, and `ForceField` will assign AM1/BCC charges as specified by the "Sage" force field. Note that the OpenFF Toolkit produces partial charges that do not depend on the input conformation of parameterized molecules. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#the-partial-charges-generated-by-the-toolkit-don-t-seem-to-depend-on-the-molecule-s-conformation-is-this-a-bug) for more information.

In [1]:
from openmm import app

from openff.toolkit import ForceField, Molecule, Topology

In [2]:
ethanol = Molecule.from_smiles("CCO")
cyclohexane = Molecule.from_smiles("C1CCCCC1")

# Load the PDB file using OpenMM and save the OpenMM Topology
pdbfile = app.PDBFile("1_cyclohexane_1_ethanol.pdb")
omm_topology = pdbfile.topology

# Create the OpenFF Topology.
off_topology = Topology.from_openmm(
    omm_topology, unique_molecules=[ethanol, cyclohexane]
)
off_topology

### Preparing an OpenFF ForceField

Once the `ForceField` class is imported, the only decision to make is which force field to use. An exhaustive list of force fields released by the Open Force Field Initiative can be found [here](from openff.toolkit.typing.engines.smirnoff import ForceField
).

Here we will use force field from the "Sage" line.

In [3]:
forcefield = ForceField("openff-2.0.0.offxml")
forcefield

### Preparing an OpenMM System

Once a force field and topology have been loaded, an `openmm.System` can be generated natively with the OpenFF Toolkit.

In [4]:
omm_system = forcefield.create_openmm_system(off_topology)
omm_system

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7ff0bb9f6ba0> >

### Preparing an Interchange object

To exports to engines other than OpenMM, we will make use of the [Interchange](https://openff-interchange.readthedocs.io/) project. There is a high-level `Interchange.from_smirnoff` function that consumes OpenFF Toolkit and ForceField objects and produces an `Interchange` object which can then be exported to formats understood by other molecular simulation engines. This extra step is needed to provide a clean interface between _applied_ parameters and engines. Note also that this step does not require an OpenMM System to be generated; `ForceField.create_openmm_system` does not need to be called to use Amber and GROMACS.

In [5]:
from openff.interchange import Interchange

interchange = Interchange.from_smirnoff(
    force_field=forcefield,
    topology=off_topology,
)
interchange.positions = pdbfile.positions
interchange

Interchange with 27 atoms, periodic topology

### Exporting to Amber and GROMACS files

Once an `Interchange` object has been constructed, its API can be used to export to files understood by GROMACS, Amber, and more.

In [6]:
# Export AMBER files.
interchange.to_prmtop("system.prmtop")
interchange.to_inpcrd("system.inpcrd")

# Export GROMACS files.
interchange.to_top("system.top")
interchange.to_gro("system.gro")

### Validating the conversion to Amber files

The Interchange project includes functions that take in an `Interchange` object and call out to simulation engines to run single-point energy calculations (with no minimization or dynamics) for the purpose of validating the export layer with each engine. Under the hood, each of these functions calls API points like those used above while converting to files understood by each engine. These rely on having each engine installed and accessible in the current `$PATH`.

In [7]:
from openff.interchange.drivers import get_amber_energies, get_openmm_energies

In [8]:
openmm_energies = get_openmm_energies(interchange)
openmm_energies.energies

{'Bond': 0.2952878475189209 <Unit('kilojoule / mole')>,
 'Angle': 155.7012939453125 <Unit('kilojoule / mole')>,
 'Torsion': 19.563228607177734 <Unit('kilojoule / mole')>,
 'vdW': 9.6586229795419 <Unit('kilojoule / mole')>,
 'Electrostatics': -15.220786403489484 <Unit('kilojoule / mole')>}

In [9]:
!cat system.inpcrd
!cat system.prmtop
!cat system.top
!cat system.gro
amber_energies = get_amber_energies(interchange)
amber_energies.energies


   27  0.0000000e+00
  12.5320000  10.8130000  10.3860000  12.3480000  10.3390000  11.8250000
  10.8800000  10.3650000  12.2440000  10.2560000  11.7390000  12.0200000
  10.4390000  12.2130000  10.5810000  11.9070000  12.1870000  10.1620000
  12.0720000  10.0880000   9.7030000  13.6000000  10.8490000  10.1430000
  12.7450000   9.3240000  11.9320000  12.9300000  10.9820000  12.4970000
  10.3240000   9.6140000  11.6690000  10.7930000  10.0880000  13.3000000
   9.1880000  11.7030000  12.2640000  10.7160000  12.4640000  12.7040000
   9.8580000  11.5710000   9.9090000  10.0430000  13.2300000  10.4740000
  11.9940000  12.4650000   9.1050000  12.4630000  12.9380000  10.7370000
  10.1720000  31.2760000  27.8660000   8.7460000  30.9610000  27.4710000
   8.7520000  30.2640000  26.2340000  10.2040000  31.8150000  28.8180000
  10.7580000  30.3560000  27.9620000  10.6610000  31.8860000  27.0990000
   8.1690000  31.8830000  27.3520000   8.2670000  30.3330000  28.2280000
   7.8260000  30.0790000  26.

{'Bond': 0.0741 <Unit('kilocalories_per_mole')>,
 'Angle': 37.2023 <Unit('kilocalories_per_mole')>,
 'Torsion': 4.6778 <Unit('kilocalories_per_mole')>,
 'vdW': 9.7231976 <Unit('kilojoule_per_mole')>,
 'Electrostatics': -15.217207999999996 <Unit('kilojoule_per_mole')>}

### Appendix: Validating the conversion to GROMACS and LAMMPS files

If GROMACS and/or LAMMPS are installed on your machine, the same comparisons can also take place with those engines. They are available via `conda` by running a command like:

```conda install "gromacs >=2021=nompi*" lammps -c conda-forge```

In [10]:
from distutils.spawn import find_executable
from pprint import pprint

from openff.interchange.drivers import get_gromacs_energies, get_lammps_energies

In [11]:
if find_executable("lmp_serial"):
    pprint(get_lammps_energies(interchange).energies)

{'Angle': <Quantity(37.213514, 'kilocalorie_per_mole')>,
 'Bond': <Quantity(0.10526246, 'kilocalorie_per_mole')>,
 'Electrostatics': <Quantity(-3.638788, 'kilocalorie_per_mole')>,
 'Torsion': <Quantity(4.675725, 'kilocalorie_per_mole')>,
 'vdW': <Quantity(2.30186395, 'kilocalorie_per_mole')>}


In [12]:
if find_executable("gmx"):
    pprint(get_gromacs_energies(interchange).energies)

Finally, there is a helper function `get_summary_data` that will attempt to run drivers of each engine. A summary reported is prepared as a Pandas `DataFrame`.

In [13]:
from openff.interchange.drivers.all import get_summary_data

get_summary_data(interchange)

,Bond,Angle,Torsion,vdW,Electrostatics
OpenMM,0.295288,155.701294,19.563229,9.658623,-15.220786
Amber,0.310034,155.654423,19.571915,9.723198,-15.217208
LAMMPS,0.440418,155.701343,19.563233,9.630999,-15.224689
